# Bài tập: Nhận diện khuôn mặt thời gian thực với FaceNet & MTCNN trên Webcam

## Các bước chính:
1. Truy cập và thu thập thông tin qua webcam bằng thư viện OpenCV
2. Tích hợp MTCNN để phát hiện khuôn mặt
3. Sử dụng FaceNet để trích xuất đặc trưng và so sánh khuôn mặt theo thời gian thực từ webcam

## Điều kiện so sánh:
- Nếu similarity >= 0.7, hiển thị "Matched"
- Nếu similarity < 0.7, hiển thị "Unknown"

## 1. Import các thư viện cần thiết

In [1]:
import cv2
import numpy as np
import torch
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
import time
from pathlib import Path

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

d:\Applications\anaconda3\envs\KNCVU\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Khởi tạo MTCNN và FaceNet

**MTCNN**: Multi-task CNN để detect và align faces  
**FaceNet**: InceptionResnetV1 pretrained trên VGGFace2 để extract 512-dim embeddings

In [2]:
mtcnn = MTCNN(
    image_size=160, margin=20,
    min_face_size=20, thresholds=[0.6, 0.7, 0.7],
    factor=0.709, post_process=True,
    device=device, keep_all=True
)

facenet = InceptionResnetV1(
    pretrained='vggface2',
    classify=False,
    device=device
).eval()

## 3. Chuẩn bị thư mục ảnh tham chiếu

Đặt ảnh khuôn mặt vào thư mục `reference_faces/` với format tên:
- Một ảnh: `Your_Name.jpg`
- Nhiều ảnh: `Your_Name_01.jpg`, `Your_Name_02.jpg`, ...

In [3]:
reference_dir = Path("reference_faces")
reference_dir.mkdir(exist_ok=True)

## 4. Hàm trích xuất embeddings từ ảnh

In [4]:
def get_face_embedding(image, mtcnn_model, facenet_model):
    if isinstance(image, np.ndarray):
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_pil = Image.fromarray(image_rgb)
    else:
        image_pil = image

    face_tensor = mtcnn_model(image_pil)

    if face_tensor is None:
        return None

    if face_tensor.ndim == 3:
        face_tensor = face_tensor.unsqueeze(0)

    with torch.no_grad():
        embedding = facenet_model(face_tensor.to(device))

    return embedding[0]


def get_face_embedding_from_tensor(face_tensor, facenet_model):
    if face_tensor is None or face_tensor.ndim < 3:
        return None

    if face_tensor.ndim == 3:
        face_tensor = face_tensor.unsqueeze(0)

    with torch.no_grad():
        embedding = facenet_model(face_tensor.to(device))

    return embedding[0]

## 5. Load và xử lý ảnh tham chiếu

In [5]:
def load_reference_faces(reference_dir, mtcnn_model, facenet_model):
    reference_embeddings = {}
    supported_formats = ['.jpg', '.jpeg', '.png', '.bmp']
    reference_path = Path(reference_dir)
    image_files = [f for f in reference_path.iterdir() if f.suffix.lower() in supported_formats]

    if not image_files:
        print(f"Không tìm thấy ảnh tham chiếu trong '{reference_dir}'")
        return reference_embeddings

    print(f"Đang load {len(image_files)} ảnh tham chiếu...")
    for img_path in image_files:
        try:
            image = Image.open(img_path).convert('RGB')
            embedding = get_face_embedding(image, mtcnn_model, facenet_model)

            if embedding is not None:
                name = img_path.stem.replace('_', ' ')
                import re
                name = re.sub(r'\s+\d+$', '', name).strip()

                if name not in reference_embeddings:
                    reference_embeddings[name] = []
                reference_embeddings[name].append(embedding)
                print(f"Loaded: {img_path.name} → {name}")
            else:
                print(f"Không detect được khuôn mặt: {img_path.name}")
        except Exception as e:
            print(f"Lỗi khi load {img_path.name}: {e}")

    print(f"\nTổng số người trong database: {len(reference_embeddings)}")
    return reference_embeddings

reference_embeddings = load_reference_faces(reference_dir, mtcnn, facenet)

Đang load 6 ảnh tham chiếu...
Loaded: Phu_Thuan_01.jpg → Phu Thuan
Loaded: Phu_Thuan_02.jpg → Phu Thuan
Loaded: Phu_Thuan_03.jpg → Phu Thuan
Loaded: Ronaldo_01.png → Ronaldo
Loaded: Ronaldo_02.png → Ronaldo
Loaded: Ronaldo_03.png → Ronaldo

Tổng số người trong database: 2


## 6. Hàm tính similarity và nhận dạng

In [6]:
def recognize_face(face_embedding, reference_embeddings, threshold=0.7):
    if not reference_embeddings:
        return "No Reference", 0.0, "Unknown"

    max_similarity = -1
    matched_name = "Unknown"

    for name, ref_embeddings in reference_embeddings.items():
        for ref_embedding in ref_embeddings:
            similarity = torch.nn.functional.cosine_similarity(face_embedding.unsqueeze(0), ref_embedding.unsqueeze(0)).item()

            if similarity > max_similarity:
                max_similarity = similarity
                matched_name = name

    status = "Matched" if max_similarity > threshold else "Unknown"

    return matched_name, max_similarity, status

## 7. Chạy nhận dạng khuôn mặt real-time từ webcam

**Các thông số:**
- **Threshold = 0.7**: Ngưỡng similarity để xác định "Matched" 
- **Detection Confidence = 0.7**: Ngưỡng MTCNN để xác nhận face hợp lệ

**Hướng dẫn:**
- Press **'q'** để thoát
- Press **'s'** để lưu ảnh hiện tại
- Khuôn mặt được nhận dạng sẽ có:
  - **Màu XANH LÁ**: Matched (similarity ≥ 0.7)
  - **Màu ĐỎ**: Unknown (similarity < 0.7)

In [7]:
def run_realtime_face_recognition(mtcnn_model, facenet_model, reference_embeddings, threshold=0.7, camera_index=0, detection_confidence=0.7):
    if not reference_embeddings:
        print("Không có reference embeddings. Vui lòng thêm ảnh vào folder 'reference_faces/'")
        return

    cap = cv2.VideoCapture(camera_index)
    if not cap.isOpened():
        print(f"Không thể mở camera index {camera_index}")
        return

    print("Camera đã mở. Nhấn 'q' để thoát, 's' để lưu ảnh.")

    frame_count = 0
    fps_time = time.time()
    fps = 0
    error_count = 0
    max_errors = 5
    capture_dir = Path("captures")
    capture_dir.mkdir(exist_ok=True)

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                error_count += 1
                if error_count >= max_errors:
                    print("Không đọc được frame từ camera sau nhiều lần thử")
                    break
                continue

            error_count = 0
            frame_count += 1
            display_frame = frame.copy()
            h, w = frame.shape[:2]

            # Convert frame sang RGB cho MTCNN
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(rgb_frame)

            # MTCNN detect faces và trả về boxes, probs, landmarks
            # Sử dụng keep_all=True để detect nhiều khuôn mặt
            boxes, probs, landmarks = mtcnn_model.detect(pil_image, landmarks=True)

            # Extract aligned face tensors từ boxes đã detect (single-pass optimization)
            # Không cần chạy detect lại, chỉ extract theo boxes
            face_tensors = mtcnn_model.extract(pil_image, boxes, save_path=None) if boxes is not None else None

            if boxes is not None:
                for idx, (box, prob, landmark) in enumerate(zip(boxes, probs, landmarks)):
                    if prob < detection_confidence:
                        continue

                    x1 = max(0, int(box[0]))
                    y1 = max(0, int(box[1]))
                    x2 = min(w-1, int(box[2]))
                    y2 = min(h-1, int(box[3]))

                    # Vẽ box ngay sau khi detect (không phụ thuộc embedding)
                    # Default màu vàng cho "detecting..."
                    box_color = (0, 255, 255)  # Vàng
                    display_name = "Detecting..."
                    similarity = 0.0
                    status = "Processing"

                    try:
                        # Lấy face tensor tương ứng với detection này
                        if face_tensors is not None:
                            if face_tensors.ndim == 3:  # Chỉ có 1 face
                                face_tensor = face_tensors if idx == 0 else None
                            else:  # Nhiều faces
                                face_tensor = face_tensors[idx] if idx < len(face_tensors) else None

                            if face_tensor is not None:
                                # Extract embedding từ face tensor
                                face_embedding = get_face_embedding_from_tensor(face_tensor, facenet_model)

                                if face_embedding is not None:
                                    # Nhận dạng
                                    name, similarity, status = recognize_face(face_embedding, reference_embeddings, threshold)

                                    if status == "Matched":
                                        box_color = (0, 255, 0)  # Xanh lá
                                        display_name = name
                                    else:
                                        box_color = (0, 0, 255)  # Đỏ
                                        display_name = "Unknown"
                                else:
                                    display_name = "Embedding Failed"
                                    box_color = (128, 128, 128)  # Xám
                        else:
                            display_name = "Alignment Failed"
                            box_color = (128, 128, 128)  # Xám

                    except Exception as e:
                        print(f"Lỗi khi xử lý face #{idx}: {e}")
                        display_name = "Error"
                        box_color = (128, 128, 128)  # Xám

                    # Vẽ bounding box
                    cv2.rectangle(display_frame, (x1, y1), (x2, y2), box_color, 2)

                    # Vẽ landmarks
                    if landmark is not None:
                        for point in landmark:
                            cv2.circle(display_frame, tuple(point.astype(int)), 2, box_color, -1)

                    # Vẽ label với tên và similarity
                    label = f"{display_name} ({similarity:.2f})" if similarity > 0 else display_name
                    label_y = y1 - 10 if y1 - 10 > 10 else y1 + 20
                    (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                    cv2.rectangle(display_frame, (x1, label_y - text_height - 5), (x1 + text_width, label_y + 5), box_color, -1)
                    cv2.putText(display_frame, label, (x1, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

                    # Vẽ status
                    if status != "Processing":
                        status_label = f"Status: {status}"
                        cv2.putText(display_frame, status_label, (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 2)

            # Tính FPS
            if frame_count % 10 == 0:
                fps = 10 / (time.time() - fps_time)
                fps_time = time.time()

            # Hiển thị FPS
            cv2.putText(display_frame, f"FPS: {fps:.1f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Hiển thị số lượng faces detected
            num_faces = len(boxes) if boxes is not None else 0
            cv2.putText(display_frame, f"Faces: {num_faces}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            cv2.imshow('Real-time Face Recognition', display_frame)

            # Xử lý phím bấm
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                print("Đang đóng camera...")
                break
            elif key == ord('s'):
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                save_path = capture_dir / f"capture_{timestamp}.jpg"
                cv2.imwrite(str(save_path), display_frame)
                print(f"Đã lưu: {save_path}")

    except KeyboardInterrupt:
        print("\nInterrupted by user")
    except Exception as e:
        print(f"Lỗi runtime: {e}")
    finally:
        cap.release()
        cv2.destroyAllWindows()
        print("Camera đã đóng")

run_realtime_face_recognition(mtcnn, facenet, reference_embeddings, threshold=0.7, camera_index=0, detection_confidence=0.7)

Camera đã mở. Nhấn 'q' để thoát, 's' để lưu ảnh.
Đang đóng camera...
Camera đã đóng
